In [1]:
import pandas as pd

merged_full_pipeline_df = pd.read_csv("merged_full_pipeline.csv")
summary_full_pipeline_df = pd.read_csv("summary_full_pipeline.csv")
merged_baseline_pipeline_df = pd.read_csv("merged_baseline_pipeline.csv")
summary_baseline = pd.read_csv("summary_baseline.csv")
merged_sparql_llm_pipeline_df = pd.read_csv("merged_sparql_llm_pipeline.csv")
summary_sparql_llm_pipeline_df = pd.read_csv("summary_sparql_llm_pipeline.csv")

In [3]:
merged_baseline_pipeline_df.columns

Index(['question', 'prompt_templates', 'prompt_examples', 'all_prompts_ranked',
       'most_similar_template', 'validation_time', 'valid_query', 'query',
       'attempts', 'initial_query_time', 'result',
       'final_query_execution_time', 'total_time', 'Unnamed: 0', 'sparql',
       'question_template', 'sparql_template', 'expected_result',
       'template_rank', 'precision', 'recall', 'exact_match',
       'sparql_template_key'],
      dtype='object')

In [12]:
import ast

valid_nonempty_validation_df = merged_baseline_pipeline_df[
    (merged_baseline_pipeline_df['valid_query']) &
    (merged_baseline_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0))
]

invalid_nonempty_validation_df = merged_baseline_pipeline_df[
    (~merged_baseline_pipeline_df['valid_query']) &
    (merged_baseline_pipeline_df['validation_time'].apply(lambda x: isinstance(ast.literal_eval(x), list) and len(ast.literal_eval(x)) > 0))
]

In [8]:
merged_baseline_pipeline_df['validation_time'][0]

'[]'

In [11]:
valid_nonempty_validation_df['attempts'].value_counts()

attempts
1    31
2     7
3     2
Name: count, dtype: int64

In [ ]:

invalid_nonempty_validation_df['attempts'].value_counts()

In [3]:
sparql_template_dict = dict(zip(summary_baseline.index, summary_baseline['sparql_template']))

In [4]:
sparql_template_dict

{0: 'PREFIX sphn: <https://www.biomedit.ch/rdf/sphn-schema/sphn/>\nPREFIX icd: <https://www.biomedit.ch/rdf/sphn-schema/sphn/icd#>\n\nSELECT (COUNT(*) AS ?numPatients) WHERE {\n  {\n    SELECT DISTINCT ?patient WHERE {\n      ?patient sphn:hasDiagnosis ?diag .\n      ?diag sphn:hasCode ?code .\n      ?code sphn:hasCodeValue icd:§Diagnosis§ .\n    }\n  }\n}\n',
 1: 'PREFIX sphn: <https://www.biomedit.ch/rdf/sphn-schema/sphn/>\nPREFIX icd: <https://www.biomedit.ch/rdf/sphn-schema/sphn/icd#>\n\nSELECT ?patient WHERE {\n  ?patient sphn:hasDiagnosis ?diag .\n  ?diag sphn:hasCode ?code .\n  ?code sphn:hasCodeValue icd:§Diagnosis§ .\n}\n',
 2: 'PREFIX sphn: <https://www.biomedit.ch/rdf/sphn-schema/sphn/>\nSELECT ?patient WHERE {\n  ?ae <http://www.w3.org/1999/02/22-rdf-syntax-ns#type>\n       sphn:AdverseEvent .\n  ?ae sphn:hasSubjectPseudoIdentifier ?patient .\n  ?ae sphn:hasSeverity ?sev .\n  FILTER(?sev IN ("§Severity§"))\n}\n',
 3: 'PREFIX sphn: <https://www.biomedit.ch/rdf/sphn-schema/sp

In [5]:
def get_template_key(sparql_template):
    for key, template in sparql_template_dict.items():
        if template == sparql_template:
            return key
    return None

In [6]:
merged_dfs = [
    merged_full_pipeline_df,
    merged_baseline_pipeline_df,
    merged_sparql_llm_pipeline_df,
]
summary_dfs = [
    summary_full_pipeline_df,
    summary_baseline,
    summary_sparql_llm_pipeline_df,
]

In [7]:
for merged_df in merged_dfs:
    merged_df["total_time"]
    merged_df["final_query_execution_time"]
    merged_df["initial_query_time"]
    # Calculate total validation time by summing the values in the 'validation_time' list for each row
    merged_df["total_validation_time"] = merged_full_pipeline_df[
        "validation_time"
    ].apply(lambda x: sum(x) if isinstance(x, list) else 0)


In [17]:
merged_sparql_llm_pipeline_df['total_validation_time'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [13]:
non_empty_validation_df = merged_baseline_pipeline_df[merged_baseline_pipeline_df['validation_time'].apply(lambda x: isinstance(x, list) and len(x) > 0)]

In [14]:
non_empty_validation_df

,question,prompt_templates,prompt_examples,all_prompts_ranked,most_similar_template,validation_time,valid_query,query,attempts,initial_query_time,...,sparql,question_template,sparql_template,expected_result,template_rank,precision,recall,exact_match,sparql_template_key,total_validation_time


In [11]:
merged_sparql_llm_pipeline_df['total_validation_time'].values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [9]:
merged_full_pipeline_df.columns

Index(['question', 'prompt_templates', 'prompt_examples', 'all_prompts_ranked',
       'most_similar_template', 'validation_time', 'valid_query', 'query',
       'attempts', 'initial_query_time', 'result',
       'final_query_execution_time', 'total_time', 'Unnamed: 0', 'sparql',
       'question_template', 'sparql_template', 'expected_result',
       'template_rank', 'precision', 'recall', 'exact_match',
       'total_validation_time', 'sparql_template_key'],
      dtype='object')